In [11]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz, process
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/recipe_df.csv')

pd.set_option('max_columns', None)

recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,9
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,176.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,203.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16


In [ ]:
#recipe_df["ingredient"].value_counts()

In [3]:
input_ing = input('¿Qué tienes hoy en la nevera?')

¿Qué tienes hoy en la nevera? huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon


In [4]:
input_ing

'huevos, pechiga de pollo, lentejas, pimiento, arroz, leche, mantequilla, harina, sal, azúcar, aceite, zanahorias, queso, bacon'

In [5]:
def input_list(string):
    input_list = []
    str_split = string.split(", ")
    for x in str_split:
        input_list.append(x)
    return input_list

#input_ingred = input_list(input_ing)

#print(input_ingred)

In [6]:
recipe_df["input"] = input_ing
recipe_df["input"] = recipe_df.apply(lambda x: input_list(x["input"]), axis=1)

In [8]:
recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients,input
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12,"[huevos, pechiga de pollo, lentejas, pimiento,..."
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,9,"[huevos, pechiga de pollo, lentejas, pimiento,..."
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16,"[huevos, pechiga de pollo, lentejas, pimiento,..."
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,176.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12,"[huevos, pechiga de pollo, lentejas, pimiento,..."
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,203.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16,"[huevos, pechiga de pollo, lentejas, pimiento,..."


In [46]:
def matched(recipe_ingred, input_ingred):
    ingred_list = ast.literal_eval(recipe_ingred)
    new_list = []
    for element in input_ingred:
        try:
            best_match = process.extract(element, ingred_list, scorer=fuzz.WRatio, limit=1)
            if best_match[0][1] >= 80:
                ingred_list.remove(best_match[0][0])
                new_list.append(best_match[0][0])
            else:
                pass
        except:
            pass
    return new_list

In [47]:
recipe_df["matched_ingred"] = recipe_df.apply(lambda x: matched(x["ingredients_list"], x["input"])
                                                  , axis=1)

In [48]:
recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients,input,matched_ingred
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",60.0,252.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12,"[huevos, pechiga de pollo, lentejas, pimiento,...","[huevos, aceite de oliva virgen extra, pimient..."
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,386.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,9,"[huevos, pechiga de pollo, lentejas, pimiento,...",[tomates secos en aceite]
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,NaN,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16,"[huevos, pechiga de pollo, lentejas, pimiento,...","[pimentón, sal, anchoas en aceite]"
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,176.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,12,"[huevos, pechiga de pollo, lentejas, pimiento,...","[pimienta, sal]"
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,203.0,0,0,0,0,0,['Primeros platos y entrantes'],principal,ensaladas,16,"[huevos, pechiga de pollo, lentejas, pimiento,...","[huevo, pechugas de pollo, pimienta, sal, acei..."


In [45]:
recipe_df.sort_values(by=["num_ingredients"], ascending=[True]).head(10)

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,tipo_plato,main_category,category,num_ingredients,input,matched_ingred
2048,https://content-recetas.lecturas.com/medio/201...,Brochetas de butifarras variadas,https://www.lecturas.com/recetas/brochetas-but...,['butifarra de setas de payés de escalivada'],20.0,NaN,1,0,0,0,1,['Segundos platos y platos principales'],principal,carnes,1,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
3798,https://content-recetas.lecturas.com/medio/201...,Músicos con fruta,https://www.lecturas.com/recetas/musicos-fruta...,"['chocolate negro fondant', 'fruta confitada']",35.0,NaN,0,0,0,0,0,['Postres'],postre,postres y dulces,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
5250,https://content-recetas.lecturas.com/medio/201...,Enrollado de sobrasada,https://www.lecturas.com/recetas/enrollado-sob...,"['pan de molde sin corteza', 'sobrasada']",10.0,NaN,0,0,0,0,0,['Aperitivos y tapas'],tapas,bocadillos y canapés,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
5327,https://content-recetas.lecturas.com/medio/201...,York a las finas hierbas,https://www.lecturas.com/recetas/york-a-finas-...,"['jamón cocido', 'queso blanco']",10.0,NaN,0,0,0,0,0,['Aperitivos y tapas'],tapas,bocadillos y canapés,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
3542,https://content-recetas.lecturas.com/medio/201...,Chocolate fundido,https://www.lecturas.com/recetas/chocolate-fun...,"['chocolate negro', 'nata líquida']",15.0,NaN,0,0,0,0,0,['Postres'],postre,postres y dulces,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
4278,https://content-recetas.lecturas.com/medio/201...,Bocaditos con nueces,https://www.lecturas.com/recetas/bocaditos-nue...,"['chocolate negro fondant', 'nueces']",15.0,NaN,0,0,0,0,0,['Postres'],postre,postres y dulces,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
4657,https://content-recetas.lecturas.com/medio/201...,Turrón de chocolate crujiente,https://www.lecturas.com/recetas/turron-chocol...,"['chocolate con leche', 'arroz inflado de choc...",45.0,NaN,0,0,0,0,0,['Postres'],postre,postres y dulces,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
3810,https://content-recetas.lecturas.com/medio/201...,Huevo de pascua,https://www.lecturas.com/recetas/huevo-pascua_...,"['chocolate negro fondant', 'fondant de color ...",40.0,NaN,1,0,0,0,1,['Postres'],postre,postres y dulces,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
5047,https://content-recetas.lecturas.com/medio/201...,Hojaldres de miel,https://www.lecturas.com/recetas/hojaldres-mie...,"['hojaldre láminas refrigeradas', 'miel de flo...",35.0,NaN,0,0,0,0,0,['Postres'],postre,pizzas y empanadas,2,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]
4276,https://content-recetas.lecturas.com/medio/201...,Rocas de almendra y chocolate,https://www.lecturas.com/recetas/rocas-almendr...,"['almendras tostadas', 'chocolate negro fondan...",15.0,NaN,0,0,0,0,0,['Postres'],postre,postres y dulces,3,"[huevos, pechiga de pollo, lentejas, pimiento,...",[]


In [ ]:
recipe_df.head(1)

In [ ]:
recipe_df["active_ingred"] = recipe_df.select_dtypes(include=['bool']).sum(axis=1)

In [ ]:
def num_missing_ingred(recipe_ingred, active_ingred):
    missing_ingred = recipe_ingred - active_ingred
    return missing_ingred

recipe_df["num_missing_ingred"] = recipe_df.apply(lambda x: num_missing_ingred(x["num_ingredients"], x["active_ingred"]), axis=1)

In [ ]:
recipe_df.head(3)

In [ ]:
def rating(match, missing):
    result = 0.75*match-0.25*missing
    return result

recipe_df["recipe_rate"] = recipe_df.apply(lambda x: rating(x["active_ingred"], x["num_missing_ingred"]), axis=1)

In [ ]:
recipe_df.head(3)

In [ ]:
filter_principal = recipe_df['main_category'] == "principal"

output_df = recipe_df[filter_principal].sort_values(by=['recipe_rate', 'num_missing_ingred'], ascending=[False, True]).head(10)

output_df

In [ ]:
output_df.columns

In [ ]:
recipe_name = output_df["recipe_name"].iloc[0]

num_missing_ingred = output_df['num_missing_ingred'].iloc[0]

recipe_time = output_df["time_preparation(min)"].iloc[0]

print(f"La mejor receta que puedes hacer con los ingredientes que has introducido es: {recipe_name}")
print(f"Te faltan {num_missing_ingred} ingredientes")
print(f"El tiempo de preparación es de {recipe_time} minutos")

In [ ]:
go_to_recipe = input('¿Quieres ir a la receta?')

def normalize(string):
    lower = string.lower()
    a,b = 'áéíóú','aeiou'
    trans = str.maketrans(a,b)
    result = lower.translate(trans)
    return result

input_answer = normalize(go_to_recipe)

In [ ]:
if input_answer == 'si':
    recipe_link = output_df["recipe_url"].iloc[0]
    def pop_recipe(url_):
        ROUTE = url_
        webbrowser.open_new(ROUTE)
    pop_recipe(recipe_link)
elif input_answer == 'no':
    recipe_link = output_df["recipe_url"].iloc[0]
    print("Genial! Éste es el link por si quieres consultar la receta más tarde: {recipe_link}")
else:
    print("Por favor, responde sí o no")